# Cantonese Dim Sum Learning Game 🥟🍵

## Overview
This is an interactive pygame-based game designed to help players learn how to order food in Cantonese at a dim sum restaurant. The game features:
- Start screen with "Start Learning" button
- Customizable canteen background, NPC, and player sprites (configure via file paths in the script)
- 10-minute countdown timer
- Dialogue box with audio playback (configure audio path in the script)
- Voice recording and Cantonese speech-to-text transcription
- **Fullscreen support with automatic aspect ratio preservation**

## Installation Requirements

```bash
pip install pygame sounddevice soundfile SpeechRecognition
```

## How to Configure Your Assets

**At the top of the main script (cell 2), you'll find a CONFIGURATION section:**

```python
# ============================================================================
# CONFIGURATION - EDIT THESE FILE PATHS TO USE YOUR OWN ASSETS
# ============================================================================

# Image Assets - Replace with your file paths
CANTEEN_BACKGROUND = None  # e.g., "assets/canteen.png"
NPC_SPRITE = None          # e.g., "assets/npc.png"
PLAYER_SPRITE = None       # e.g., "assets/player.png"
TABLE_SPRITE = None        # e.g., "assets/table.png"

# Audio Assets - Replace with your file paths
DIALOGUE_AUDIO = None      # e.g., "assets/dialogue.mp3"
WELCOME_AUDIO = None       # e.g., "assets/welcome.mp3" - NPC greeting before sitting
```

**Example with actual paths:**
```python
CANTEEN_BACKGROUND = "C:/Users/User/Documents/images/canteen.png"
NPC_SPRITE = "C:/Users/User/Documents/images/waiter.png"
PLAYER_SPRITE = "C:/Users/User/Documents/images/customer.png"
TABLE_SPRITE = "C:/Users/User/Documents/images/table.png"
DIALOGUE_AUDIO = "C:/Users/User/Documents/audio/greeting.mp3"
WELCOME_AUDIO = "C:/Users/User/Documents/audio/welcome.mp3"
```

Simply replace `None` with your file paths (use forward slashes `/` or double backslashes `\\\\`).

## Features

### 1. **Scene Management**
   - Start screen with "Start Learning" button
   - Main game scene with your custom assets

### 2. **Asset Configuration**
   - Set file paths in the script for:
     - Canteen background (PNG/JPG)
     - NPC sprite (PNG with transparency recommended)
     - Player sprite (PNG with transparency recommended)
     - Table sprite (PNG with transparency recommended)
     - Dialogue audio (MP3/WAV) - main conversation
     - Welcome audio (MP3/WAV) - NPC greeting before sitting

### 3. **Aspect Ratio & Fullscreen**
   - **Press F11** to toggle fullscreen mode
   - **Press ESC** to exit fullscreen
   - Game maintains 4:3 aspect ratio (1024x768) in all modes
   - Background images are scaled to cover the screen while maintaining their aspect ratio
   - Black letterboxing on sides or top/bottom if window doesn't match game aspect ratio

### 4. **Timer**
   - 10-minute countdown timer in the top-right corner
   - Changes to red when less than 1 minute remains

### 5. **Dialogue System**
   - Dialogue box displays NPC conversation
   - Play button to hear the dialogue audio (MP3)
   - Example dialogue: "几位，饮咩茶？我们有铁观音，普洱、红茶"

### 6. **Voice Recording & Transcription**
   - Record button to capture player's spoken response
   - Automatic Cantonese speech-to-text transcription
   - Display transcribed text below the recording area

## Cantonese Audio-to-Text Transcription Methods (No GPU Required)

### Method 1: **Google Speech Recognition API** (Used in this script)
**Pros:**
- Free (with usage limits)
- Supports Cantonese (yue-Hant-HK)
- No GPU required
- Easy to implement

**Cons:**
- Requires internet connection
- Limited accuracy for informal Cantonese
- Usage limits apply

**Implementation:** Already included in the script using `speech_recognition` library with language code `"yue-Hant-HK"`

### Method 2: **Azure Speech Service**
**Pros:**
- Better accuracy than Google
- Supports Hong Kong Cantonese (zh-HK)
- Professional-grade transcription
- No GPU required (cloud-based)

**Cons:**
- Requires Azure account (free tier available)
- Costs money after free tier
- Requires internet

**Setup:**
```python
pip install azure-cognitiveservices-speech

import azure.cognitiveservices.speech as speechsdk

speech_config = speechsdk.SpeechConfig(
    subscription="YOUR_KEY", 
    region="YOUR_REGION"
)
speech_config.speech_recognition_language="zh-HK"  # Hong Kong Cantonese
```

### Method 3: **Wit.ai (Facebook/Meta)**
**Pros:**
- Free
- Supports Cantonese
- No GPU required
- Good for conversational AI

**Cons:**
- Requires API key
- Internet connection required
- Less accurate than Azure

**Setup:**
```python
pip install wit

from wit import Wit
client = Wit('YOUR_ACCESS_TOKEN')
client.speech('audio.wav', {'Content-Type': 'audio/wav'})
```

### Method 4: **OpenAI Whisper (CPU Mode)**
**Pros:**
- Open-source
- Supports multiple languages including Cantonese
- Can run offline
- Very accurate

**Cons:**
- Slower on CPU (but still works without GPU)
- Larger model size
- Higher memory usage

**Setup:**
```python
pip install openai-whisper

import whisper
model = whisper.load_model("base")  # Use "base" or "small" for CPU
result = model.transcribe("audio.wav", language="zh")  # Chinese (includes Cantonese)
print(result["text"])
```

### Method 5: **Vosk (Offline)**
**Pros:**
- Completely offline
- No GPU required
- Free and open-source
- Fast on CPU

**Cons:**
- Cantonese model availability may be limited
- Requires downloading language models
- May have lower accuracy than cloud services

**Setup:**
```python
pip install vosk

from vosk import Model, KaldiRecognizer
import wave

model = Model("path/to/cantonese-model")
wf = wave.open("audio.wav", "rb")
rec = KaldiRecognizer(model, wf.getframerate())
```

## Recommended Approach

For your use case, I recommend **starting with Google Speech Recognition** (already implemented) because:
1. It's free and easy to use
2. No setup complexity
3. Decent Cantonese support
4. No GPU required

If you need better accuracy, **upgrade to Azure Speech Service** which offers superior Cantonese recognition for Hong Kong dialect.

For offline capability, use **OpenAI Whisper in CPU mode** - it's slower but works without internet and has excellent accuracy.

## Usage Instructions

1. **Configure your assets** at the top of the script (file paths)
2. **Run the script**
3. Click "**Start Learning**" on the start screen
4. The NPC will say "Find any empty seat"
5. **Click on the table** in the bottom-right corner to sit down
6. After sitting, the main dialogue will begin
7. **Press F11** for fullscreen mode
8. Click the **play button (🔊)** to hear the audio
9. Click "**🎤 Record**" to start recording your response
10. Click "**⏹ Stop**" to stop recording
11. The system will automatically transcribe your Cantonese speech
12. Your transcribed text appears in the answer box

## Customization

### To change dialogue and audio during gameplay:
```python
# Modify this line in the handle_events method:
self.set_dialogue("Your dialogue text here", "path/to/audio.mp3")
```

### To add multiple dialogue sequences:
Create a list of dialogues and cycle through them based on player responses.

## Notes
- Make sure your microphone is connected and working
- The default transcription uses Google's API (requires internet)
- All images maintain their aspect ratio when scaled
- The game window can be resized or set to fullscreen (F11)
- Black bars (letterboxing) appear if the window aspect ratio differs from 4:3
- You can switch to other transcription methods by modifying the `transcribe_audio` method in the `AudioRecorder` class

In [10]:
import pygame
import sys
import os
from pathlib import Path
from datetime import timedelta
import threading

# Audio recording and speech recognition libraries
# You'll need to install these: pip install pygame sounddevice soundfile SpeechRecognition
import sounddevice as sd
import soundfile as sf
import speech_recognition as sr

# ============================================================================
# CONFIGURATION - EDIT THESE FILE PATHS TO USE YOUR OWN ASSETS
# ============================================================================

# Image Assets - Replace with your file paths
CANTEEN_BACKGROUND = "temp png file/background.png"  # e.g., "assets/canteen.png"
NPC_SPRITE = "temp png file/npc.png"          # e.g., "assets/npc.png"
PLAYER_SPRITE = "temp png file/player.png"       # e.g., "assets/player.png"
TABLE_SPRITE = "temp png file/table.png"                    # e.g., "assets/table.png"

# Audio Assets - Replace with your file paths
DIALOGUE_AUDIO = "audio file/1.mp3"      # e.g., "assets/dialogue.mp3"
WELCOME_AUDIO = "audio file/2.mp3"       # e.g., "assets/welcome.mp3" - NPC greeting before sitting

# Tea response audio files - NPC responses for each tea type
TEA_AUDIO_TIEGUANYIN = "audio file/tea_tieguanyin.mp3"  # "Okay, one Tieguanyin"
TEA_AUDIO_XIANGPIAN = "audio file/tea_xiangpian.mp3"    # "Okay, one jasmine tea"
TEA_AUDIO_PUER = "audio file/tea_puer.mp3"              # "Okay, one Pu-erh"
TEA_AUDIO_HONGCHA = "audio file/tea_hongcha.mp3"        # "Okay, one black tea"

# Menu and UI Assets
MENU_IMAGE = "temp png file/menu.png"                    # Menu image (94x136)
CLICK_ICON = "temp png file/click_icon.png"              # Click icon to show when NPC is nearby

# Example with actual paths:
# CANTEEN_BACKGROUND = "C:/Users/User/Documents/images/canteen.png"
# NPC_SPRITE = "C:/Users/User/Documents/images/waiter.png"
# PLAYER_SPRITE = "C:/Users/User/Documents/images/customer.png"
# DIALOGUE_AUDIO = "C:/Users/User/Documents/audio/greeting.mp3"

# ============================================================================

# Initialize Pygame
pygame.init()
pygame.mixer.init()

# Constants
GAME_WIDTH = 1536   # Base game resolution width
GAME_HEIGHT = 864   # Base game resolution height
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
DARK_GRAY = (100, 100, 100)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (100, 149, 237)

# Timer settings
GAME_TIME = 15  # 15 seconds for player response

# Tea keywords for recognition
TEA_KEYWORDS = {
    "tieguanyin": "tieguanyin",
    "鐵觀音": "tieguanyin",
    "铁观音": "tieguanyin",
    "jasmine": "xiangpian",
    "jasmine tea": "xiangpian",
    "香片": "xiangpian",
    "pu-erh": "puer",
    "puer": "puer",
    "普洱": "puer",
    "black tea": "hongcha",
    "紅茶": "hongcha",
    "红茶": "hongcha"
}

class Button:
    def __init__(self, x, y, width, height, text, color, text_color):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.color = color
        self.text_color = text_color
        self.font = pygame.font.Font(None, 36)
        
    def draw(self, screen):
        pygame.draw.rect(screen, self.color, self.rect)
        pygame.draw.rect(screen, BLACK, self.rect, 2)
        text_surface = self.font.render(self.text, True, self.text_color)
        text_rect = text_surface.get_rect(center=self.rect.center)
        screen.blit(text_surface, text_rect)
        
    def is_clicked(self, pos):
        return self.rect.collidepoint(pos)

class AudioRecorder:
    def __init__(self):
        self.is_recording = False
        self.recording = []
        self.sample_rate = 44100
        self.temp_file = "temp_recording.wav"
        
    def start_recording(self):
        self.is_recording = True
        self.recording = []
        
        def record():
            with sd.InputStream(samplerate=self.sample_rate, channels=1, callback=self.audio_callback):
                while self.is_recording:
                    sd.sleep(100)
        
        self.record_thread = threading.Thread(target=record)
        self.record_thread.start()
        
    def audio_callback(self, indata, frames, time, status):
        if self.is_recording:
            self.recording.append(indata.copy())
            
    def stop_recording(self):
        self.is_recording = False
        if hasattr(self, 'record_thread'):
            self.record_thread.join()
        
        if self.recording:
            import numpy as np
            recording_array = np.concatenate(self.recording, axis=0)
            sf.write(self.temp_file, recording_array, self.sample_rate)
            return self.temp_file
        return None
    
    def transcribe_audio(self, audio_file):
        """
        Transcribe Cantonese audio to text using Google Speech Recognition
        Note: This requires internet connection and supports Cantonese (yue-Hant-HK)
        Returns tuple: (english_display_text, original_chinese_text)
        """
        recognizer = sr.Recognizer()
        
        try:
            with sr.AudioFile(audio_file) as source:
                audio = recognizer.record(source)
                
            # Using Google Speech Recognition with Cantonese language code
            # Language code: yue-Hant-HK (Cantonese, Traditional Chinese, Hong Kong)
            original_text = recognizer.recognize_google(audio, language="yue-Hant-HK")
            
            print(f"[DEBUG] Original transcription: '{original_text}'")
            
            # Convert Chinese characters to English-safe representation for display
            english_text = self.convert_to_english_display(original_text)
            
            print(f"[DEBUG] English display: '{english_text}'")
            
            # Return both versions: display text and original text for keyword detection
            return (english_text, original_text)
            
        except sr.UnknownValueError:
            return ("Could not understand audio", "")
        except sr.RequestError as e:
            return (f"Service error: {e}", "")
        except Exception as e:
            return (f"Error: {e}", "")
    
    def convert_to_english_display(self, chinese_text):
        """
        Convert Chinese text to English-safe display format
        Maps common tea-related terms to English
        """
        # Common Cantonese dim sum phrases mapping
        translation_map = {
            "鐵觀音": "Tieguanyin",
            "铁观音": "Tieguanyin",
            "香片": "Jasmine tea",
            "普洱": "Pu-erh",
            "紅茶": "Black tea",
            "红茶": "Black tea",
            "一位": "One person",
            "两位": "Two people",
            "兩位": "Two people",
            "三位": "Three people",
            "四位": "Four people",
            "好嘅": "Okay",
            "好的": "Okay",
            "唔該": "Thank you",
            "謝謝": "Thank you",
            "多謝": "Thank you"
        }
        
        # Try to translate common phrases
        result = chinese_text
        for chinese, english in translation_map.items():
            result = result.replace(chinese, english)
        
        # If still contains Chinese characters, provide a fallback
        # Check if any Chinese characters remain
        if any('\u4e00' <= char <= '\u9fff' for char in result):
            # Still has Chinese characters
            return f"[Recognized: {len(result)} characters]"
        
        return result

class Game:
    def __init__(self):
        # Setup display with aspect ratio preservation
        self.game_width = GAME_WIDTH
        self.game_height = GAME_HEIGHT
        self.aspect_ratio = GAME_WIDTH / GAME_HEIGHT
        
        # Get display info for fullscreen
        display_info = pygame.display.Info()
        self.display_width = display_info.current_w
        self.display_height = display_info.current_h
        
        # Start in windowed mode
        self.fullscreen = False
        self.screen = pygame.display.set_mode((GAME_WIDTH, GAME_HEIGHT), pygame.RESIZABLE)
        
        # Create game surface that maintains aspect ratio
        self.game_surface = pygame.Surface((GAME_WIDTH, GAME_HEIGHT))
        
        pygame.display.set_caption("Cantonese Learning Game - Dim Sum Ordering")
        self.clock = pygame.time.Clock()
        self.running = True
        self.state = "start"  # start, game, end
        
        # Timer
        self.time_remaining = GAME_TIME
        self.start_time = None
        self.timer_active = False  # Timer only starts after player sits
        
        # Fonts
        self.title_font = pygame.font.Font(None, 72)
        self.dialogue_font = pygame.font.Font(None, 32)
        self.timer_font = pygame.font.Font(None, 48)
        
        # Buttons
        self.start_button = Button(
            GAME_WIDTH // 2 - 150, GAME_HEIGHT // 2 - 50,
            300, 100, "Start Learning", GREEN, BLACK
        )
        
        # Load game images from file paths
        self.canteen_bg = None
        self.npc_image = None
        self.player_image = None
        self.table_image = None
        self.menu_image = None
        self.click_icon = None
        self.load_assets()
        
        # Player state
        self.player_seated = False  # Player not seated initially
        
        # Table position (bottom-right corner, moved more to bottom-right)
        self.table_rect = pygame.Rect(GAME_WIDTH - 400, GAME_HEIGHT - 260, 300, 250)
        
        # Menu state
        self.show_menu = False
        self.menu_rect = None
        
        # NPC movement system
        self.npc_path_points = [
            (GAME_WIDTH - 550, GAME_HEIGHT - 480),  # Starting point (current NPC position)
            (GAME_WIDTH - 700, GAME_HEIGHT - 480),  # Move left
            (GAME_WIDTH - 700, GAME_HEIGHT - 200),  # Move up
            (150, GAME_HEIGHT - 200),               # Move far left
            (150, GAME_HEIGHT - 500),               # Move up left
            (400, GAME_HEIGHT - 500),               # Move right
            (400, GAME_HEIGHT - 200),               # Move down
            (GAME_WIDTH - 700, GAME_HEIGHT - 200),  # Back to right area
        ]
        self.npc_current_path_index = 0
        self.npc_position = list(self.npc_path_points[0])  # Current NPC position [x, y]
        self.npc_speed = 5.0  # Pixels per frame
        self.npc_at_player = False
        self.npc_wait_time = 0
        self.npc_wait_duration = 3000  # 3 seconds in milliseconds
        self.npc_moving = False  # Only start moving after timer ends
        
        # Audio
        self.current_dialogue_audio = None
        self.dialogue_audio_path = DIALOGUE_AUDIO
        self.welcome_audio_path = WELCOME_AUDIO
        
        # Tea audio paths
        self.tea_audio_paths = {
            "tieguanyin": TEA_AUDIO_TIEGUANYIN,
            "xiangpian": TEA_AUDIO_XIANGPIAN,
            "puer": TEA_AUDIO_PUER,
            "hongcha": TEA_AUDIO_HONGCHA
        }
        
        # Recording
        self.recorder = AudioRecorder()
        self.record_button = Button(
            GAME_WIDTH - 380, 345,
            240, 64, "Record Answer", RED, WHITE
        )
        self.is_recording = False
        self.transcribed_text = ""
        
        # Tea selection tracking
        self.tea_selected = None
        self.waiting_for_tea_choice = False
        
        # Dialogue
        self.current_dialogue = ""
        self.dialogue_lines = []
        
    def load_assets(self):
        """Load all assets from the file paths specified at the top of the script"""
        # Load canteen background
        if CANTEEN_BACKGROUND and os.path.exists(CANTEEN_BACKGROUND):
            try:
                self.canteen_bg = pygame.image.load(CANTEEN_BACKGROUND)
                print(f"✓ Loaded canteen background: {CANTEEN_BACKGROUND}")
            except Exception as e:
                print(f"✗ Failed to load canteen background: {e}")
        else:
            print("⚠ No canteen background specified or file not found")
            
        # Load NPC sprite
        if NPC_SPRITE and os.path.exists(NPC_SPRITE):
            try:
                self.npc_image = pygame.image.load(NPC_SPRITE)
                print(f"✓ Loaded NPC sprite: {NPC_SPRITE}")
            except Exception as e:
                print(f"✗ Failed to load NPC sprite: {e}")
        else:
            print("⚠ No NPC sprite specified or file not found")
            
        # Load player sprite
        if PLAYER_SPRITE and os.path.exists(PLAYER_SPRITE):
            try:
                self.player_image = pygame.image.load(PLAYER_SPRITE)
                print(f"✓ Loaded player sprite: {PLAYER_SPRITE}")
            except Exception as e:
                print(f"✗ Failed to load player sprite: {e}")
        else:
            print("⚠ No player sprite specified or file not found")
            
        # Load table sprite
        if TABLE_SPRITE and os.path.exists(TABLE_SPRITE):
            try:
                self.table_image = pygame.image.load(TABLE_SPRITE)
                print(f"✓ Loaded table sprite: {TABLE_SPRITE}")
            except Exception as e:
                print(f"✗ Failed to load table sprite: {e}")
        else:
            print("⚠ No table sprite specified or file not found")
        
        # Load menu image
        if MENU_IMAGE and os.path.exists(MENU_IMAGE):
            try:
                self.menu_image = pygame.image.load(MENU_IMAGE)
                print(f"✓ Loaded menu image: {MENU_IMAGE}")
            except Exception as e:
                print(f"✗ Failed to load menu image: {e}")
        else:
            print("⚠ No menu image specified or file not found")
        
        # Load click icon
        if CLICK_ICON and os.path.exists(CLICK_ICON):
            try:
                self.click_icon = pygame.image.load(CLICK_ICON)
                print(f"✓ Loaded click icon: {CLICK_ICON}")
            except Exception as e:
                print(f"✗ Failed to load click icon: {e}")
        else:
            print("⚠ No click icon specified or file not found")
    
    def toggle_fullscreen(self):
        """Toggle between fullscreen and windowed mode while maintaining aspect ratio"""
        self.fullscreen = not self.fullscreen
        
        if self.fullscreen:
            self.screen = pygame.display.set_mode((self.display_width, self.display_height), 
                                                   pygame.FULLSCREEN)
        else:
            self.screen = pygame.display.set_mode((GAME_WIDTH, GAME_HEIGHT), pygame.RESIZABLE)
    
    def get_scaled_rect(self):
        """Calculate the rect for the game surface to maintain aspect ratio"""
        screen_width, screen_height = self.screen.get_size()
        screen_aspect = screen_width / screen_height
        
        if screen_aspect > self.aspect_ratio:
            # Screen is wider than game - letterbox on sides
            scaled_height = screen_height
            scaled_width = int(scaled_height * self.aspect_ratio)
            x_offset = (screen_width - scaled_width) // 2
            y_offset = 0
        else:
            # Screen is taller than game - letterbox on top/bottom
            scaled_width = screen_width
            scaled_height = int(scaled_width / self.aspect_ratio)
            x_offset = 0
            y_offset = (screen_height - scaled_height) // 2
            
        return pygame.Rect(x_offset, y_offset, scaled_width, scaled_height)
    
    def scale_mouse_pos(self, mouse_pos):
        """Convert screen mouse position to game surface coordinates"""
        scaled_rect = self.get_scaled_rect()
        
        # Check if mouse is within the game area
        if not scaled_rect.collidepoint(mouse_pos):
            return None
            
        # Convert to game coordinates
        x = (mouse_pos[0] - scaled_rect.x) * GAME_WIDTH / scaled_rect.width
        y = (mouse_pos[1] - scaled_rect.y) * GAME_HEIGHT / scaled_rect.height
        
        return (int(x), int(y))
        
    def load_dialogue_audio(self, audio_path):
        """Load and play dialogue audio"""
        try:
            self.dialogue_audio_path = audio_path
            pygame.mixer.music.load(audio_path)
            pygame.mixer.music.play()
        except Exception as e:
            print(f"Failed to load audio: {e}")
    
    def set_dialogue(self, text, audio_path=None):
        """Set the dialogue text and optional audio"""
        self.current_dialogue = text
        # Wrap text for display - use smaller width for 80% sized box
        # Box width is 384, minus padding on both sides (10*2) = 364
        self.dialogue_lines = self.wrap_text(text, pygame.font.Font(None, 28), 360)
        
        if audio_path:
            self.load_dialogue_audio(audio_path)
    
    def wrap_text(self, text, font, max_width):
        """Wrap text to fit within max_width"""
        words = text.split(' ')
        lines = []
        current_line = []
        
        for word in words:
            test_line = ' '.join(current_line + [word])
            if font.size(test_line)[0] <= max_width:
                current_line.append(word)
            else:
                if current_line:
                    lines.append(' '.join(current_line))
                current_line = [word]
        
        if current_line:
            lines.append(' '.join(current_line))
        
        return lines
    
    def draw_start_screen(self):
        """Draw the start screen"""
        self.game_surface.fill(WHITE)
        
        # Title
        title = self.title_font.render("Cantonese Dim Sum", True, BLACK)
        subtitle = self.dialogue_font.render("Learn to Order in Cantonese!", True, DARK_GRAY)
        
        title_rect = title.get_rect(center=(GAME_WIDTH // 2, 200))
        subtitle_rect = subtitle.get_rect(center=(GAME_WIDTH // 2, 280))
        
        self.game_surface.blit(title, title_rect)
        self.game_surface.blit(subtitle, subtitle_rect)
        
        # Start button
        self.start_button.draw(self.game_surface)
        
        # Instruction for fullscreen
        instruction = pygame.font.Font(None, 24).render("Press F11 for fullscreen", True, DARK_GRAY)
        instruction_rect = instruction.get_rect(center=(GAME_WIDTH // 2, GAME_HEIGHT - 50))
        self.game_surface.blit(instruction, instruction_rect)
        
    def draw_game_screen(self):
        """Draw the main game screen"""
        # Background (canteen) - maintain aspect ratio
        if self.canteen_bg:
            # Scale background to cover the game surface while maintaining aspect ratio
            bg_rect = self.canteen_bg.get_rect()
            scale_x = GAME_WIDTH / bg_rect.width
            scale_y = GAME_HEIGHT / bg_rect.height
            scale = max(scale_x, scale_y)  # Use max to cover entire surface
            
            new_width = int(bg_rect.width * scale)
            new_height = int(bg_rect.height * scale)
            scaled_bg = pygame.transform.scale(self.canteen_bg, (new_width, new_height))
            
            # Center the background
            x_offset = (GAME_WIDTH - new_width) // 2
            y_offset = (GAME_HEIGHT - new_height) // 2
            
            self.game_surface.blit(scaled_bg, (x_offset, y_offset))
        else:
            self.game_surface.fill((245, 222, 179))  # Wheat color as default
        
        # Draw NPC at current position (maintain aspect ratio) - same size as player, 120 height
        npc_x, npc_y = int(self.npc_position[0]), int(self.npc_position[1])
        
        if self.npc_image:
            npc_rect = self.npc_image.get_rect()
            # Scale to height of 120 (same as player: 150*0.8=120), maintain aspect ratio
            scale_factor = 120 / npc_rect.height
            npc_width = int(npc_rect.width * scale_factor)
            npc_scaled = pygame.transform.scale(self.npc_image, (npc_width, 120))
            self.game_surface.blit(npc_scaled, (npc_x, npc_y))
        else:
            pygame.draw.rect(self.game_surface, BLUE, (npc_x, npc_y, 96, 120))
            npc_label = self.dialogue_font.render("NPC", True, WHITE)
            self.game_surface.blit(npc_label, (npc_x + 30, npc_y + 50))
        
        # Draw Table (bottom-right corner)
        if self.table_image:
            # Scale table while maintaining aspect ratio
            table_rect_img = self.table_image.get_rect()
            scale_factor = min(self.table_rect.width / table_rect_img.width, 
                             self.table_rect.height / table_rect_img.height)
            table_width = int(table_rect_img.width * scale_factor)
            table_height = int(table_rect_img.height * scale_factor)
            table_scaled = pygame.transform.scale(self.table_image, (table_width, table_height))
            self.game_surface.blit(table_scaled, (self.table_rect.x, self.table_rect.y))
        else:
            # Draw placeholder table
            pygame.draw.rect(self.game_surface, WHITE, self.table_rect)
            pygame.draw.rect(self.game_surface, BLACK, self.table_rect, 3)
            if not self.player_seated:
                # Show "Click to sit" message
                sit_text = pygame.font.Font(None, 28).render("Click to sit", True, DARK_GRAY)
                sit_rect = sit_text.get_rect(center=self.table_rect.center)
                self.game_surface.blit(sit_text, sit_rect)
            
        # Draw Player (only if seated, maintain aspect ratio, 80% size)
        if self.player_seated:
            if self.player_image:
                player_rect = self.player_image.get_rect()
                # Scale to height of 120 (150*0.8=120), maintain aspect ratio
                scale_factor = 120 / player_rect.height
                player_width = int(player_rect.width * scale_factor)
                player_scaled = pygame.transform.scale(self.player_image, (player_width, 120))
                # Position player at the LEFT side of the table (sitting position)
                player_x = self.table_rect.left - player_width - 15  # Left of table with small gap
                player_y = self.table_rect.centery - 100  # Vertically centered with table
                self.game_surface.blit(player_scaled, (player_x, player_y))
            else:
                # Draw placeholder player - 80% size and positioned at left of table
                player_rect = pygame.Rect(self.table_rect.left - 112, self.table_rect.centery - 60, 96, 120)
                pygame.draw.rect(self.game_surface, GREEN, player_rect)
                player_label = self.dialogue_font.render("Player", True, WHITE)
                self.game_surface.blit(player_label, (player_rect.x + 12, player_rect.y + 48))
        
        # Draw menu on table (if timer has ended and tea is selected)
        if self.show_menu and self.player_seated:
            menu_x = self.table_rect.centerx - 47  # Center on table (94/2 = 47)
            menu_y = self.table_rect.centery - 68  # Center on table (136/2 = 68)
            
            if self.menu_image:
                # Menu is 94x136, draw it directly
                self.game_surface.blit(self.menu_image, (menu_x, menu_y))
                self.menu_rect = pygame.Rect(menu_x, menu_y, 94, 136)
            else:
                # Fallback: draw placeholder
                self.menu_rect = pygame.Rect(menu_x, menu_y, 94, 136)
                pygame.draw.rect(self.game_surface, WHITE, self.menu_rect)
                pygame.draw.rect(self.game_surface, BLACK, self.menu_rect, 2)
                menu_label = pygame.font.Font(None, 20).render("Menu", True, BLACK)
                self.game_surface.blit(menu_label, (menu_x + 25, menu_y + 60))
            
            # Draw click icon if NPC is at player position
            if self.npc_at_player and self.click_icon:
                # Position icon above menu
                icon_x = menu_x + 47 - self.click_icon.get_width() // 2
                icon_y = menu_y - self.click_icon.get_height() - 10
                self.game_surface.blit(self.click_icon, (icon_x, icon_y))
            elif self.npc_at_player:
                # Fallback: draw text prompt
                prompt = pygame.font.Font(None, 24).render("Click to order!", True, RED)
                self.game_surface.blit(prompt, (menu_x - 10, menu_y - 30))
        
        # Timer (top right corner) - only show if active
        if self.timer_active:
            self.draw_timer()
        
        # Dialogue box
        self.draw_dialogue_box()
        
        # Recording box
        self.draw_recording_box()
        
    def draw_timer(self):
        """Draw the countdown timer"""
        # Only show timer if it's active (after player sits)
        if not self.timer_active:
            return
            
        if self.start_time:
            elapsed = pygame.time.get_ticks() - self.start_time
            self.time_remaining = max(0, GAME_TIME - elapsed // 1000)
            
            # Check if time is up
            if self.time_remaining == 0:
                self.timer_active = False  # Hide timer
                self.show_menu = True  # Show menu on table
                self.npc_moving = True  # Start NPC movement
                
                # Handle tea timeout if no tea selected
                if self.waiting_for_tea_choice and not self.tea_selected:
                    self.handle_tea_timeout()
        
        seconds = self.time_remaining
        timer_text = f"{seconds:02d}s"
        
        # Timer background - positioned at top right (more to the right)
        timer_rect = pygame.Rect(GAME_WIDTH - 220, 20, 180, 80)
        pygame.draw.rect(self.game_surface, WHITE, timer_rect)
        pygame.draw.rect(self.game_surface, BLACK, timer_rect, 3)
        
        # Timer text - red when less than 6 seconds
        timer_surface = self.timer_font.render(timer_text, True, BLACK if self.time_remaining > 5 else RED)
        timer_text_rect = timer_surface.get_rect(center=timer_rect.center)
        self.game_surface.blit(timer_surface, timer_text_rect)
        
    def draw_dialogue_box(self):
        """Draw the dialogue display box"""
        # Position below timer in top-right area (80% size)
        box_rect = pygame.Rect(GAME_WIDTH - 420, 120, 384, 160)  # 480*0.8=384, 200*0.8=160
        pygame.draw.rect(self.game_surface, WHITE, box_rect)
        pygame.draw.rect(self.game_surface, BLACK, box_rect, 3)
        
        title_font = pygame.font.Font(None, 28)
        title = title_font.render("Dialogue:", True, BLACK)
        self.game_surface.blit(title, (GAME_WIDTH - 410, 130))
        
        # Dialogue text - ensure it stays within box boundaries
        y_offset = 160
        dialogue_font = pygame.font.Font(None, 26)  # Slightly smaller font
        max_lines = 3  # Limit number of lines to fit in box
        for i, line in enumerate(self.dialogue_lines[:max_lines]):
            if y_offset + 26 > box_rect.bottom - 10:  # Check if text exceeds box
                break
            text_surface = dialogue_font.render(line, True, BLACK)
            self.game_surface.blit(text_surface, (GAME_WIDTH - 410, y_offset))
            y_offset += 28
        
        # Play audio button (if audio is loaded)
        if self.dialogue_audio_path:
            play_button = Button(GAME_WIDTH - 130, 130, 90, 35, "Play", BLUE, WHITE)
            play_button.draw(self.game_surface)
            
    def draw_recording_box(self):
        """Draw the recording box"""
        # Position below dialogue box (80% size)
        box_rect = pygame.Rect(GAME_WIDTH - 420, 300, 384, 160)  # 480*0.8=384, 200*0.8=160
        pygame.draw.rect(self.game_surface, WHITE, box_rect)
        pygame.draw.rect(self.game_surface, BLACK, box_rect, 3)
        
        # Title
        title = pygame.font.Font(None, 28).render("Your Answer:", True, BLACK)  # Smaller font
        self.game_surface.blit(title, (GAME_WIDTH - 410, 310))
        
        # Record button (80% size)
        record_button_small = Button(GAME_WIDTH - 380, 345, 240, 64, 
                                     "Stop" if self.is_recording else "Record", 
                                     GREEN if self.is_recording else RED, WHITE)
        record_button_small.draw(self.game_surface)
        
        # Update main record button position for click detection
        self.record_button.rect = pygame.Rect(GAME_WIDTH - 380, 345, 240, 64)
        
        # Show transcribed text
        if self.transcribed_text:
            # Wrap transcribed text
            wrapped_transcription = self.wrap_text(self.transcribed_text, 
                                                   pygame.font.Font(None, 22), 350)
            y_offset = 420
            for line in wrapped_transcription[:2]:  # Show max 2 lines
                text_surface = pygame.font.Font(None, 22).render(line, True, DARK_GRAY)
                self.game_surface.blit(text_surface, (GAME_WIDTH - 410, y_offset))
                y_offset += 25
    
    def detect_tea_keyword(self, text):
        """Detect tea keywords in transcribed text"""
        if not text:
            return None
        
        text_lower = text.lower()
        print(f"[DEBUG] Checking text for tea keywords: '{text}'")
        print(f"[DEBUG] Lowercase version: '{text_lower}'")
        
        # Check each keyword (both original case and lowercase)
        for keyword, tea_type in TEA_KEYWORDS.items():
            keyword_lower = keyword.lower()
            # Check both original text and lowercase text
            if keyword in text or keyword_lower in text_lower:
                print(f"[DEBUG] FOUND tea keyword: '{keyword}' -> {tea_type}")
                return tea_type
        
        print(f"[DEBUG] No tea keyword found in: '{text}'")
        return None
    
    def handle_tea_selection(self, tea_type):
        """Handle tea selection and play corresponding NPC response"""
        self.tea_selected = tea_type
        self.waiting_for_tea_choice = False
        
        # Get tea name in English
        tea_names = {
            "tieguanyin": "Tieguanyin",
            "xiangpian": "jasmine tea",
            "puer": "Pu-erh",
            "hongcha": "black tea"
        }
        tea_name = tea_names.get(tea_type, "jasmine tea")
        
        # All tea types use the same format: "Okay, one [tea name]"
        dialogue_text = f"Okay, one {tea_name}"
        
        # Get audio path for the tea type
        audio_path = self.tea_audio_paths.get(tea_type)
        
        self.set_dialogue(dialogue_text, audio_path)
    
    def handle_tea_timeout(self):
        """Handle timeout - default to jasmine tea"""
        if not self.tea_selected:
            print("Timeout - defaulting to jasmine tea")
            self.handle_tea_selection("xiangpian")
    
    def update_npc_movement(self):
        """Update NPC position along the path"""
        if not self.npc_moving:
            return
        
        # Check if NPC is waiting at player
        if self.npc_at_player:
            current_time = pygame.time.get_ticks()
            if current_time - self.npc_wait_time >= self.npc_wait_duration:
                # Finish waiting, continue moving
                self.npc_at_player = False
                self.npc_current_path_index = (self.npc_current_path_index + 1) % len(self.npc_path_points)
            return
        
        # Get target position
        target = self.npc_path_points[self.npc_current_path_index]
        
        # Calculate direction
        dx = target[0] - self.npc_position[0]
        dy = target[1] - self.npc_position[1]
        distance = (dx**2 + dy**2) ** 0.5
        
        # Check if reached target
        if distance < self.npc_speed:
            self.npc_position = list(target)
            
            # Check if this is the player position (index 0)
            if self.npc_current_path_index == 0:
                self.npc_at_player = True
                self.npc_wait_time = pygame.time.get_ticks()
            else:
                # Move to next waypoint
                self.npc_current_path_index = (self.npc_current_path_index + 1) % len(self.npc_path_points)
        else:
            # Move towards target
            self.npc_position[0] += (dx / distance) * self.npc_speed
            self.npc_position[1] += (dy / distance) * self.npc_speed
    
    def handle_events(self):
        """Handle pygame events"""
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.running = False
            
            # Handle window resize
            if event.type == pygame.VIDEORESIZE:
                if not self.fullscreen:
                    self.screen = pygame.display.set_mode((event.w, event.h), pygame.RESIZABLE)
            
            # Handle keyboard events
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_F11:
                    self.toggle_fullscreen()
                elif event.key == pygame.K_ESCAPE and self.fullscreen:
                    self.toggle_fullscreen()
                    
            if event.type == pygame.MOUSEBUTTONDOWN:
                # Scale mouse position to game coordinates
                game_pos = self.scale_mouse_pos(pygame.mouse.get_pos())
                
                if game_pos is None:
                    continue  # Click was outside game area
                
                if self.state == "start":
                    if self.start_button.is_clicked(game_pos):
                        self.state = "game"
                        # Don't start timer yet - wait until player sits
                        # Show welcome dialogue before sitting
                        self.set_dialogue("Find any empty seat", WELCOME_AUDIO)
                        
                elif self.state == "game":
                    # Check if player clicks on table to sit
                    if not self.player_seated and self.table_rect.collidepoint(game_pos):
                        self.player_seated = True
                        # Start timer when player sits
                        self.start_time = pygame.time.get_ticks()
                        self.timer_active = True
                        self.waiting_for_tea_choice = True
                        # Set initial dialogue after sitting
                        self.set_dialogue("How many people? What tea would you like? We have Tieguanyin, jasmine tea, Pu-erh, and black tea.", 
                                        DIALOGUE_AUDIO)
                    
                    # Only allow other interactions if player is seated
                    if self.player_seated:
                        # Check if menu is clicked (only when NPC is at player and menu is showing)
                        if self.show_menu and self.npc_at_player and self.menu_rect:
                            if self.menu_rect.collidepoint(game_pos):
                                print("[INFO] Menu clicked! Player can place order.")
                                # You can add order functionality here later
                                # For now, just show a message
                                self.set_dialogue("What would you like to order?", None)
                        
                        # Check play audio button (updated position)
                        play_button_rect = pygame.Rect(GAME_WIDTH - 130, 130, 90, 35)
                        if play_button_rect.collidepoint(game_pos) and self.dialogue_audio_path:
                            pygame.mixer.music.load(self.dialogue_audio_path)
                            pygame.mixer.music.play()
                        
                        # Check record button
                        if self.record_button.is_clicked(game_pos):
                            if not self.is_recording:
                                self.is_recording = True
                                self.recorder.start_recording()
                                self.transcribed_text = "Recording..."
                            else:
                                self.is_recording = False
                                audio_file = self.recorder.stop_recording()
                                if audio_file:
                                    # Transcribe in a separate thread to avoid blocking
                                    def transcribe():
                                        self.transcribed_text = "Transcribing..."
                                        # Get both display text and original text
                                        result = self.recorder.transcribe_audio(audio_file)
                                        
                                        # Handle tuple return (display_text, original_text)
                                        if isinstance(result, tuple):
                                            display_text, original_text = result
                                            self.transcribed_text = display_text
                                            
                                            # Use original Chinese text for keyword detection
                                            if self.waiting_for_tea_choice and not self.tea_selected:
                                                detected_tea = self.detect_tea_keyword(original_text)
                                                if detected_tea:
                                                    print(f"[SUCCESS] Detected tea: {detected_tea}")
                                                    self.handle_tea_selection(detected_tea)
                                                else:
                                                    print(f"[INFO] No tea keyword detected in: '{original_text}'")
                                        else:
                                            # Fallback for error messages (single string)
                                            self.transcribed_text = result
                                    
                                    threading.Thread(target=transcribe).start()
    
    def update(self):
        """Update game state"""
        # Update NPC movement
        self.update_npc_movement()
    
    def run(self):
        """Main game loop"""
        while self.running:
            self.handle_events()
            
            # Update game state
            self.update()
            
            # Draw to game surface
            if self.state == "start":
                self.draw_start_screen()
            elif self.state == "game":
                self.draw_game_screen()
            
            # Scale and draw game surface to screen with letterboxing
            self.screen.fill(BLACK)  # Black bars for letterboxing
            scaled_rect = self.get_scaled_rect()
            scaled_surface = pygame.transform.scale(self.game_surface, 
                                                    (scaled_rect.width, scaled_rect.height))
            self.screen.blit(scaled_surface, scaled_rect)
            
            pygame.display.flip()
            self.clock.tick(60)
        
        pygame.quit()
        sys.exit()

# Run the game
if __name__ == "__main__":
    game = Game()
    game.run()

✓ Loaded canteen background: temp png file/background.png
✓ Loaded NPC sprite: temp png file/npc.png
✓ Loaded player sprite: temp png file/player.png
✓ Loaded table sprite: temp png file/table.png
✓ Loaded menu image: temp png file/menu.png
✓ Loaded click icon: temp png file/click_icon.png
[DEBUG] Original transcription: '紅茶'
[DEBUG] English display: 'Black tea'
[DEBUG] Checking text for tea keywords: '紅茶'
[DEBUG] Lowercase version: '紅茶'
[DEBUG] FOUND tea keyword: '紅茶' -> hongcha
[SUCCESS] Detected tea: hongcha
[DEBUG] Original transcription: '紅茶'
[DEBUG] English display: 'Black tea'
[DEBUG] Checking text for tea keywords: '紅茶'
[DEBUG] Lowercase version: '紅茶'
[DEBUG] FOUND tea keyword: '紅茶' -> hongcha
[SUCCESS] Detected tea: hongcha


SystemExit: 

## Quick Start Installation

Run this cell first to install all required packages:

In [ ]:
# Installation command - run this in your terminal or uncomment and run here
# !pip install pygame sounddevice soundfile SpeechRecognition

# Verify installations
try:
    import pygame
    print("✓ pygame installed")
except ImportError:
    print("✗ pygame not installed - run: pip install pygame")

try:
    import sounddevice as sd
    print("✓ sounddevice installed")
except ImportError:
    print("✗ sounddevice not installed - run: pip install sounddevice")

try:
    import soundfile as sf
    print("✓ soundfile installed")
except ImportError:
    print("✗ soundfile not installed - run: pip install soundfile")

try:
    import speech_recognition as sr
    print("✓ SpeechRecognition installed")
except ImportError:
    print("✗ SpeechRecognition not installed - run: pip install SpeechRecognition")

✗ pygame not installed - run: pip install pygame
✗ sounddevice not installed - run: pip install sounddevice
✗ soundfile not installed - run: pip install soundfile
✗ SpeechRecognition not installed - run: pip install SpeechRecognition


## Alternative: Simplified Version with OpenAI Whisper (Offline, CPU-based)

If you want an offline solution with better Cantonese accuracy, here's a modified version using Whisper:

In [ ]:
# Alternative transcription method using OpenAI Whisper
# Install with: pip install openai-whisper

def transcribe_audio_whisper(audio_file):
    """
    Transcribe Cantonese audio using OpenAI Whisper (works offline, CPU-only)
    More accurate than Google Speech Recognition for Cantonese
    """
    try:
        import whisper
        
        # Load model - use "base" or "small" for CPU, "medium" or "large" for better accuracy
        # First run will download the model
        print("Loading Whisper model...")
        model = whisper.load_model("base")  # Options: tiny, base, small, medium, large
        
        # Transcribe with Cantonese language hint
        print("Transcribing audio...")
        result = model.transcribe(audio_file, language="zh")  # Chinese (includes Cantonese)
        
        return result["text"]
        
    except ImportError:
        return "Whisper: pip install openai-whisper"
    except Exception as e:
        return f"Error: {e}"

# To use this in the game, replace the transcribe_audio method in AudioRecorder class:
# def transcribe_audio(self, audio_file):
#     return transcribe_audio_whisper(audio_file)

## Quick Example: Setting Up File Paths

Here's how to configure your assets in the script:

1. **Find the CONFIGURATION section** at the top of cell 2
2. **Replace the file paths** with your own:

```python
# Instead of:
CANTEEN_BACKGROUND = None

# Use your actual file path:
CANTEEN_BACKGROUND = "C:/Users/User/Documents/images/restaurant.png"
```

**Important Notes:**
- Use forward slashes `/` or double backslashes `\\\\` in paths
- Set to `None` if you don't have a file yet (game will use default placeholders)
- All images will maintain their aspect ratio when scaled
- Transparent PNG files work best for NPC and Player sprites

**Keyboard Controls:**
- **F11** - Toggle fullscreen
- **ESC** - Exit fullscreen (when in fullscreen mode)